<a href="https://colab.research.google.com/github/atharvas/utils/blob/master/bs4_bullshit_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [0]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import datetime
import time
import random
from io import StringIO


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
profDF = pd.read_csv("/gdrive/My Drive/SPRING_2019/CS225H/proj3/graybook-cleaned.csv")

## Helper Functions

In [0]:
def getProfMetrics(link):
  url = base_url + link
  source = requests.get(url).text
  soup = BeautifulSoup(source, 'lxml')
  ratings = soup.find_all("div", class_="grade")#.text
  return [i.text.strip() for i in ratings]

getProfMetrics("/ShowRatings.jsp?tid=1462973")

['10\nGrade Received (Optional)\n\n\n\nSelect\nA+\nA\nA-\nB+\nB\nB-\nC+\nC\nC-\nD+\nD\nD-\nF\nDrop/Withdrawal\nIncomplete\nNot sure yet\nRather not say\nAudit/No Grade']

In [0]:
def getProfRatings(name):
  base_url = "https://www.ratemyprofessors.com"
  url = base_url + "/search.jsp?queryBy=teacherName&queryoption=HEADER&query=" + name.replace(" ", "+") + "&" + "facetSearch=true"
  source = requests.get(url).text
  soup = BeautifulSoup(source, 'lxml')
  link = soup.find("ul", class_="listings")
  if link is None:
    return None
  profs = link.find_all("a")
  link = profs[len(profs) - 1]['href']
  prof_metrics = getProfMetrics(link)
  prof_metrics.append(name)
  retDF = pd.DataFrame(prof_metrics).T
  retDF.columns = ["Rating", "Again", "Difficulty", "Name"] 
  return retDF

getProfRatings("Nerissa Brown")

,Rating,Again,Difficulty,Name
0,4.3,N/A,1.0,Nerissa Brown


In [0]:
def reverse_name(name):
  parts = name.split(", ")
  parts[1] = parts[1].split(" ")[0]
  new_name = parts[1] + " " + parts[0]  
  return new_name

In [0]:
cleanDF = profDF[["Employee Name", "Primary Job Title", "Primary College", "Primary Department", "Salary"]]

mask = cleanDF['Primary Job Title'] != "???"
mask2 = cleanDF['Primary Department'] != "???"
mask3 = cleanDF['Primary Job Title'] != "???"

cleanDF = cleanDF.loc[mask & mask2 & mask3]
cleanDF['Employee Name'] = cleanDF['Employee Name'].apply(lambda x: reverse_name(x))
profs = cleanDF['Employee Name'].tolist()

In [0]:
sleep_list = [0.5, 0.75, 1, 1.25, 1.50, 1.75, 2.0]
huge_df = list()

for prof in profs:
  print(prof)
  time.sleep(random.choice(sleep_list)) # wait some seconds before making each request to prevent DDOSing their site.
  rating = getProfRatings(prof)
  if rating is not None:
    huge_df.append(rating)

huge_df = pd.concat(huge_df)
huge_df.to_csv("muh_data.csv")

Manu Edakara
John Quarton
Valeri Werpetinski
Ahmed Abdel-Khalik
Waqar Ahmed
Vikrant Anand
Spencer Anderson
Gerlando Augusto Sampaio Franco de Lima
Nerissa Brown
Timothy Brown
